## Problema de regresión

### Descripción del problema

En este problema vamos a trabajar con el conjunto de datos _Airfoil Self-Noise_, el cuál ha sido proporcionado por la **NASA** y es resultado de haber realizado un conjunto de pruebas aerodinámicas y acústicas en un túnel de viento sobre perfiles alares de dos y tres dimensiones.

El conjunto de datos está compuesto por 1503 filas de 6 elementos. Cada elemento es un valor numérico real. Los datos de las 5 primeras columnas se corresponden con los datos de entrada, y se corresponden con los siguientes atributos:

1. Frecuencia, medida en Hz.
2. Ángulo de ataque (ángulo que forman la cuerda geométrica de un perfil alar con la dirección del aire incidente), medida en grados.
3. Longitud de la cuerda del perfil alar, medida en metros.
4. Free-stream velocity, in meters per second. 
5. Suction side displacement thickness, in meters. 

La salida se corresponde con la última columna, y representa:
6. Nivel de presión sonora, medida en decibelios. 